# Ejercicios de evaluación de logaritmo discreto

Ejercicio. Los parámetros de un criptosistema de ElGamal son $p = 211$ y $g = 3$, es decir, el criptosistema está diseñado en el cuerpo $F_{211}$ = $Z_{211}$ y tomamos como generador de $F^∗_{211}$, $g = 3$. La clave pública empleada es $3^a = 109 \mod 211$. Descifra el criptograma $(154, d \mod 211)$. Para calcular los logaritmos discretos necesarios emplea dos de los métodos descritos en la teoría.


## Solución  

La función que descifra el mensaje viene dada por: 
$$D_a(x,y) = y x^{-a} = y x^{p-1-a}$$
que a partir de las hipótesis de teoría consistiría en calcular: 

$$D_a(K, m) = k m ^{p-1-a}$$

In [2]:
p = 211
k = 156
d = 75577392
m = mod(d,p) 
print(f"Is prime p = {p}  ? The answer is: {is_prime(211)}.")
print(f"El criptograma a descifrar es ({k}, {m})")


Is prime p = 211  ? The answer is: True.
El criptograma a descifrar es (156, 146)


Puesto que como se indica el enunciado la clave pública empleada es 
$$3^a = 109 \mod 211,$$
la clave del ejercicio resido entonces en calcular el logaritmo discreto para conocer $a$. 
El logaritmo discreto podrá ser calculado con el algoritmo de `Shanks` y de `Silver-Pohling-Hellman`. 

In [3]:
def Shanks(h,b,n):
    """
    Calcula el logaritmo discreto x: x = log_b(h)
    n es el orden del grupo
    b es la base 
    h = b^x
    """
    f = ceil(sqrt(n-1))
    potencia_iesima_b = 1
    table = []
    for i in range(f):
        table.append((i,potencia_iesima_b))
        potencia_iesima_b= mod(potencia_iesima_b*b, p)
        
    inverso_b = power_mod(b,n-f-1, p)
    h_0 = h
    for i in range(f):
        # comprobamos si existe un j tal que (j,h_i) esté en la tabla
        for j in range(f):
            if(j,h_0) in table: 
                return j+i*f
        # no se ha encontrado 
        h_0 = h_0*inverso_b
    #no existe logaritmo discreto
    return 0

def SacaJ(e,r,p):
    for j in range(p):
        if e == r[j]:
            return j
    return 1

def SilverPoligHellman(b, n, h):
    l = list(factor(n))
    r = len(l)
    r_list = []
    m = []
    p_super_e = []
    for i in range(r): # recorremos cada productorio 
        r_list.append([])
        (p,e) = l[i]
        p_super_e.append(p**e)
        p_aux = p
        acc = 0
        for j in range(e):
            r_ij = b^(j*n/p) %n
            r_list[i].append(r_ij)
        
        for k in range(e):
            y_k = h/(b^acc)
            x_k = SacaJ(y_k^(n/p_aux), r_list[i], e)
            acc += x_k*p^k
            p_aux*=p
        m.append(acc)
    return crt(m, p_super_e)
     
    

In [4]:
b = 3
h = mod(109,p)
n = p-1

print("Logaritmo discreto de Shanks: ", Shanks(h,b,p))
print(f"Logaritmo discreto de sage: {discrete_log(h,b,n)}") # Comparamos nuestro resultado con el de Sage 
#print("Logaritmo discreto de Silver Polig Hellman: ", SilverPoligHellman(b, n, h))

Logaritmo discreto de Shanks:  96
Logaritmo discreto de sage: 96
Logaritmo discreto de Silver Polig Hellman:  96


In [5]:
# Desciframos el mensaje
a = Shanks(h,b,p) # podría haberse utilizado el algoritmo de SilverPohlingHellman
mensaje = k * pow(m , p-1-a)
print("El mensaje descifrado es: ", m)

El mensaje descifrado es:  146
